In [1]:
from pytket import Circuit, OpType
from pytket.circuit.display import render_circuit_jupyter
import numpy as np


def _string_to_indices(bitstring: str) -> list:
    """
    Helper function: Given an input bitstring returns a list of qubit indices
    on which to apply X gates in step 1.
    """
    index_list = []

    for count, bit in enumerate(bitstring):
        if bit == "0":
            index_list.append(count)

    return index_list

![oracles](oracles.png)

See this paper -> https://www.diva-portal.org/smash/get/diva2:1214481/FULLTEXT01.pdf

These are "trivial oracles" they simply mark a chosen basis state. For an actual application of Grover's we would need a more interesting oracle

In [2]:
def grover_oracle_circuit(bitstring: str) -> Circuit:
    """
    Builds a circuit which performs the grover oracle. Takes a bitstring representation
    of a basis state as input and produces the circuit for marking the basis state in the superposition.
    """
    n_qubits = len(bitstring)
    circ = Circuit(n_qubits)

    x_indices = _string_to_indices(
        bitstring
    )  # List of indices where we apply a Pauli X gate

    for j in x_indices:  # First layer of Pauli gates
        circ.X(j)

    circ.add_gate(OpType.CnZ, list(range(n_qubits)))

    for k in x_indices:  # Second layer of Pauli gates
        circ.X(k)

    return circ

In [4]:
render_circuit_jupyter(grover_oracle_circuit('1010'))

In [5]:
from pytket.extensions.qiskit import AerStateBackend

test_circ = Circuit(4)

for qubit in range(test_circ.n_qubits):
    test_circ.H(qubit)
    
test_circ.append(grover_oracle_circuit('0000'))
render_circuit_jupyter(test_circ)

backend = AerStateBackend()
c_circ = backend.get_compiled_circuit(test_circ)
state = backend.run_circuit(c_circ).get_state()

print(np.round(state, 2))

[-0.25-0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j
  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j  0.25+0.j
  0.25+0.j  0.25-0.j]


In [7]:
def grover_diffusion_circuit(n_qubits: int) -> Circuit:
    """
    Constructs a circuit that returns the Grover diffusion operator for a given number of qubits.
    """

    circ = Circuit(n_qubits)

    for j in range(n_qubits):
        circ.H(j)
        circ.X(j)

    circ.add_gate(OpType.CnZ, list(range(n_qubits)))

    for k in range(n_qubits):
        circ.X(k)
        circ.H(k)

    return circ

In [8]:
render_circuit_jupyter(grover_diffusion_circuit(4))

In [9]:
def build_grover_circuit(bitstring: str, n_iters: int) -> Circuit:
    """
    Constructs a Grover's circuit for a specified computational basis state and number of iterations.
    """
    n_qubits = len(bitstring)
    circ = Circuit(n_qubits)

    # prepare uniform superposition
    for j in range(n_qubits):
        circ.H(j)

    # repeat the marking and diffusion steps n_iters times
    for k in range(n_iters):
        circ.append(grover_oracle_circuit(bitstring))
        circ.append(grover_diffusion_circuit(n_qubits))

    # measure all qubits
    circ.measure_all()

    return circ

In [11]:
render_circuit_jupyter(build_grover_circuit("1010", 2))

In [12]:
# experiment parameters
binary_string = "10011"
n_shots = 1000
n_iters = 3

grover_circuit_exp = build_grover_circuit(binary_string, n_iters=n_iters)


# calculate optimal number of iterations for a given problem size
def optimial_iteration_number(n_qubits: int, m_solutions=1):
    """
    Returns the theoretically optimal number of Grover iterations given the
    size of the problem and the number of solutions.
    """
    N = 2**n_qubits
    ans = np.floor(np.pi / 4 * np.sqrt(N / m_solutions) - 1 / 2)
    return int(ans)


n_qubits = len(binary_string)
opt = optimial_iteration_number(n_qubits)
print(f"The optimal number of Grover iterations is {opt}.")

The optimal number of Grover iterations is 3.


In [16]:
from pytket.extensions.qiskit import AerBackend

backend = AerBackend()  # initalise noiseless Aer simulator

compiled_circ = backend.get_compiled_circuit(grover_circuit_exp)
print(f"Number of quantum gates = {compiled_circ.n_gates}.")
print(f"Number of CX gates = {compiled_circ.depth_by_type(OpType.CX)}.")

render_circuit_jupyter(compiled_circ)
handle = backend.process_circuit(compiled_circ, n_shots=n_shots)
grover_result = backend.get_result(handle)


from pytket.

Number of quantum gates = 468.
Number of CX gates = 210.


Counter({(1, 0, 0, 1, 1): 912, (1, 0, 1, 1, 1): 6, (1, 1, 1, 0, 0): 6, (0, 0, 0, 1, 0): 5, (0, 1, 1, 0, 1): 5, (0, 0, 1, 0, 1): 4, (0, 0, 1, 1, 0): 4, (0, 0, 1, 1, 1): 4, (0, 1, 0, 1, 1): 4, (1, 0, 0, 0, 0): 4, (1, 0, 1, 1, 0): 4, (1, 1, 0, 1, 1): 4, (0, 0, 0, 0, 1): 3, (0, 1, 0, 0, 1): 3, (0, 1, 1, 1, 0): 3, (1, 0, 0, 1, 0): 3, (1, 1, 0, 1, 0): 3, (1, 1, 1, 1, 1): 3, (0, 0, 0, 0, 0): 2, (0, 0, 1, 0, 0): 2, (0, 1, 0, 1, 0): 2, (0, 1, 1, 0, 0): 2, (1, 0, 1, 0, 1): 2, (1, 1, 0, 0, 0): 2, (1, 1, 0, 0, 1): 2, (1, 1, 1, 1, 0): 2, (0, 1, 0, 0, 0): 1, (0, 1, 1, 1, 1): 1, (1, 0, 0, 0, 1): 1, (1, 0, 1, 0, 0): 1})


In [14]:
from plotting import plot_results

plot_results(grover_result, dark_mode=True, y_limit=1000)
print(grover_result.get_counts())